# Overview

This tutorial will give you an overview how to get from a DICOM dump to a processed Dataset with segmentations.

abbreviations:
POI: Point of interest

Steps:

(1) Dicom export to BIDS dataset

(2) ~~Inter-scan image registration​.~~

(2.1) ~~Rigide Movement correction with automatic Spine POIs~~

(2.2) ~~Deformable Movement~~

(3) Stitching

(3.1) ~~Stitching with rigid movement compensation. (From 2.1)~~

(3.2) ~~Stitching with deformable movement compensation. (From 2.2)~~


(4) Segmentation TotalVibeSegmentator, Spineps ...

(5) ~~MR Deformable Registration (From 2.1,2.2)~~

(6) ~~Water Fat Swap detection in VIBE and MEVIBE~~


## 1 Dicom export to BIDS dataset

Short overview:

A BIDS dataset is a file naming convection.

The following rules should be known and weakly enforced:

- A dataset folder should start with 'dataset-{YOUR-NAME}'
- The next level folder are:
  - rawdata: for all imaging data.
  - derivative: for all generated data, like segmentation.
A file should look like:

sub-{Subject name}_ses-{Session}_{key}-{value}*_{format}.{filetype}
- Subject name: Unique identifier 
- Session: Session id. Optional if there is only one session
- Any number of key-values. Keys are unique. The defined keys are here: https://bids-specification.readthedocs.io/en/stable/appendices/entities.html . Our tool enforces a certain order. See tutorial_BIDS_files.ipynb
- format: type of acquisition like ct, T2w, VIBE, MPRage
Do not use '_' in any key or values. 

See https://bids-specification.readthedocs.io/en/stable/ for detailed description what BIDS ist.

In [ ]:
from TPTBox.core.bids_files import entities_keys, formats

print('Known formats:\n','\n'.join(formats))
print()
print()
print("Order of keys we enforce:\n", '\n'.join(entities_keys.keys()))


This function extracts a dicom folder to a BIDS-like Niffty folder.

The names are created like this: DICOM:Key is given dicom key
      
'dataset-{NAME}/rawdate/sub-{DICOM:PatientID}/ses-{DICOM:StudyDate}/{format}/sub-{DICOM:PatientID}_ses-{DICOM:StudyDate}_sequ-{DICOM:SeriesNumber}_acq-{sag|ax|cor|iso}_{format}.nii.gz'

and a .json, where the and DICOM-Keys are saved.

To get {format} we use string matching and the dicom "SeriesDescription" key. As this is a free text this will not always work. Than we default to "mr" and you have to manually rename them.


For very large dataset you can use make_subject_chunks = n [int]. Than we put a additional folder with the first n letters between rawdata and the sub- folder.

In [ ]:
from pathlib import Path

from TPTBox.core.dicom.dicom_extract import extract_dicom_folder

path_to_dicom_dataset = "TODO"
dataset_name = 'example-name'

path_to_dicom_dataset = "/media/data/robert/datasets/dicom_example/VR-DICOM/"
dataset_name = 'VR-DICOM2'
target_folder = Path(path_to_dicom_dataset).parent
dataset = target_folder / f"dataset-{dataset_name}"
extract_dicom_folder(Path(path_to_dicom_dataset), dataset,use_session=True,n_cpu=1)

In [ ]:
from pathlib import Path

path_to_dicom_dataset = "/media/data/robert/datasets/dicom_example/VR-DICOM/"
dataset_name = 'VR-DICOM2'
target_folder = Path(path_to_dicom_dataset).parent
dataset = target_folder / f"dataset-{dataset_name}"

We have tool that automat scans Bids folders an creates a grouped dictionary, where you can pick out the relevant.

In [ ]:
from TPTBox import BIDS_FILE, BIDS_Global_info
from TPTBox.core.bids_constants import sequence_splitting_keys

print("if one of the values of these keys is diffrent, than it is considered a other sequence:", sequence_splitting_keys)
print("sub will alway split")

print("Lets search for candidate for merging. For this we have to remove the sequ-key from sequence_splitting_keys")
my_splitting_keys = sequence_splitting_keys.copy()
my_splitting_keys.remove("sequ")
my_splitting_keys.append("part")

bgi = BIDS_Global_info(dataset,["rawdata","derivative"],sequence_splitting_keys=my_splitting_keys)
stitching_candidate:list[BIDS_FILE] = []
epsilon = 0.2
for name, subj in bgi.iter_subjects():
    print('Subject identifier',name)
    q = subj.new_query()
    #Filter by some rules
    q.flatten()
    q.filter_filetype('nii.gz')
    q.unflatten()
    for fam in q.loop_dict():
        print(fam)
        for key, file_list in fam.items():
            if key == "mr":
                continue
            if len(file_list) == 1:
                continue
            # This code is only an example, where we group images with the same orientation and zoom, so we know what are potential stitching targets.
            # We use _format key as the initial split, so T1w and T2w will not be stiched
            matching_group = []
            for files in range(len(file_list)):
                f1 = file_list[files]
                if f1 is None:
                    continue
                grid1 = f1.get_grid_info()
                if grid1 is None:
                    continue
                current_group = [f1]  # Start a new group with the current file
                for j in range(files + 1, len(file_list)):
                    f2 = file_list[j]
                    if f2 is None:
                        continue
                    grid2 = f2.get_grid_info()
                    if grid2 is None:
                        continue
                    # Check if orientation matches
                    if grid1.orientation == grid2.orientation:
                        # Check if zoom is within the tolerance
                        zoom_diff = [abs(z1 - z2) for z1, z2 in zip(grid1.zoom, grid2.zoom,strict=False)]
                        if all(diff <= epsilon for diff in zoom_diff):
                            current_group.append(f2)
                            file_list[j] = None # type: ignore
                # Add the group if it has more than one file
                if len(current_group) > 1:
                    stitching_candidate.append(current_group)
for files in stitching_candidate:
    print(files)

# 3 Stitching  
Torax/Fullbody images are often in chunks. We can stich them with the stitching function

In [ ]:
from concurrent.futures import ProcessPoolExecutor

from TPTBox import to_nii
from TPTBox.stitching import stitching

derivative_folder = "derivative_stiched"

def process_files(files):
    files = sorted(files)  # noqa: PLW2901
    sequ: str = (files[0].get("sequ", "") + "-" if "sequ" in files[0].info else "") + "stiched"  # type: ignore
    out_name = files[0].get_changed_path("nii.gz", info={"sequ": sequ}, parent=derivative_folder)
    if not out_name.exists():
        stitching(*files, out=out_name, is_seg=False, is_ct=files[0].bids_format == "ct", dtype=to_nii(files[0]).dtype)
        nii = to_nii(out_name)
        nii.apply_crop_(nii.compute_crop())
        nii.save(out_name)
# Test
process_files(stitching_candidate[0])
# Execute the loop in parallel using a ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    executor.map(process_files, stitching_candidate)

# 3 Segmentation  

Note: by default we do not install Deep-learning stuff.

Install:

```pip install SPINEPS ruamel.yaml configargparse```

trouble shouting: nnunetv2==2.4.2


### TotalVibeSegmentator

https://arxiv.org/abs/2406.00125

https://github.com/robert-graf/TotalVibeSegmentator


In [ ]:
from TPTBox import BIDS_FILE
from TPTBox.segmentation import run_totalvibeseg

# run_totalvibeseg
# You can alos use a string/Path if you want to set the path yourself.
dataset = "/media/data/robert/datasets/dicom_example/dataset-VR-DICOM2/"
in_file = BIDS_FILE(f"{dataset}/derivative_stiched/sub-111168222/T2w/sub-111168222_sequ-301-stiched_acq-ax_part-water_T2w.nii.gz",dataset)
out_file = in_file.get_changed_path("nii.gz","msk",parent="derivative",info={"seg":"TotalVibeSegmentator","mod":in_file.bids_format})
run_totalvibeseg(in_file,out_file,True)

## spineps

Spineps can segment spine images in a instance and semantic mask. Running automatic over a dataset is very opinionated, what to segment. 
TODO: make a way to manully define output paths

https://github.com/Hendrik-code/spineps/tree/main

In [ ]:
# If your model is BIDS compliant you can auto run spineps
from TPTBox.segmentation import run_spineps_all
#run_spineps_all(dataset)


In [ ]:
# Pick a fitting model:
from spineps.models import modelid2folder_instance, modelid2folder_semantic

print('Available Semantic Models',modelid2folder_semantic())
print('Available Instance Models',modelid2folder_instance())

print(modelid2folder_semantic().keys())
print(modelid2folder_instance().keys())
dataset = "/media/data/robert/datasets/dicom_example/dataset-VR-DICOM2"
file_path = f"{dataset}/derivative_stiched/sub-111168223/T2w/sub-111168223_sequ-401-stiched_acq-sag_part-inphase_T2w.nii.gz"
#file_path = f"{dataset}/derivative_stiched/sub-111168223/T2w/sub-111168223_sequ-201-stiched_acq-ax_part-inphase_T2w.nii.gz"

model_semantic = "t2w"
model_instance = "instance"
derivative_name = "derivative"


In [ ]:
from TPTBox.segmentation.spineps import run_spineps_single

#With 'ignore_compatibility_issues = True' you can force to rund
out_paths = run_spineps_single(
    file_path,
    dataset=dataset,
    model_semantic=model_semantic,
    model_instance=model_instance,
    derivative_name=derivative_name,
    ignore_compatibility_issues=False,)
print(out_paths)